# Motivation
Trying pure, simple and straigntforward fastai v2 process.
Fastai2 (v.2.6.3) full path from training to testing and submission, with the augmentation during training.

In [ ]:
from fastai.vision.all import *
import pandas as pd
%matplotlib inline

# Create Training-Validation data loader

In [ ]:
train_path='../input/histopathologic-cancer-detection/train/'
train_df = pd.read_csv('../input/histopathologic-cancer-detection/train_labels.csv')
# for interactive DEBUG: reduce amount of train images : train_df = train_df[:1024]

batch_tfms = [ *aug_transforms(max_rotate=0, max_zoom=1., max_warp=0), Normalize.from_stats(*imagenet_stats) ]

dls = ImageDataLoaders.from_df(train_df, path=train_path, suff='.tif', 
      batch_tfms=batch_tfms, bs=128, shuffle=True)

In [ ]:
# try this to see dataloader working

# dls.train.show_batch(max_n=12)
# dls.valid.show_batch(max_n=12)

# Load pre-trained model and fine tune it

In [ ]:
learn = vision_learner(dls, densenet121, path='.', 
    loss_func=FocalLoss(),  
    metrics=[BalancedAccuracy(), RocAucBinary()] ) # this dataset is unbalanced (40/60)

In [ ]:
learn.fine_tune(4)

In [ ]:
learn.export()

# Create Testing data loader

In [ ]:
test_path='../input/histopathologic-cancer-detection/test/'
test_df = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')
# for interactive DEBUG: test_df = test_df[:12]

# new loader for the sake of path, which if different from the training path
tdls = ImageDataLoaders.from_df(test_df, path=test_path, suff='.tif',
       batch_tfms=batch_tfms, shuffle=False)
tst_dl = tdls.test_dl(test_df) 

In [ ]:
# try it working !!! here I need to make sure aug is working, seems like not
# tst_dl.show_batch(max_n=12)

# Test and output Submission file

In [ ]:
preds, y = learn.get_preds(dl=tst_dl)

preds_f1 = torch.softmax(preds, dim=1)[:, 1]
lb_df = pd.DataFrame({ 'label' : preds_f1 })
test_df.label = lb_df.label
test_df.to_csv(f'submission_HST.csv', header=True, index=False)
test_df.head()

# Have a nice day!
Was it easy? Upvote and comment, please :)